In [1]:
import utils
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"  # or choose another backend if you wish
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, DatetimeTickFormatter
from bokeh.models.tools import HoverTool
from bokeh.io import output_notebook
import sklearn
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import datetime
import keras
from keras.models import Sequential, Model
from keras.layers import Embedding, Flatten, Dense, Dropout, Input

Using plaidml.keras.backend backend.


In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
weather_data = utils.load_weather_data()
actual_load = utils.load_pal_data()
print(weather_data.shape)
print(actual_load.shape)

(5537, 6)
(5513, 3)


In [4]:
merged = actual_load.join(weather_data, how='inner')
print(merged.shape)
if merged.isnull().values.any():
    print('Null values detected in dataset!')

(5513, 9)


In [5]:
merged = merged.drop(columns=['pal_min', 'pal_max'])

In [6]:
merged = merged.sample(frac=1)  # to be changed later
merged.head()

,pal_mean,weekday,week,month,PRCP,TMAX,TMIN
2009-02-27,5893.711765,4,9,2,0.04,61.0,43.0
2014-06-03,6732.903082,1,23,6,0.12,87.0,65.0
2013-08-21,7813.025424,2,34,8,0.00,90.0,72.0
2013-07-10,8876.531724,2,28,7,0.00,85.0,75.0
2017-01-27,5773.755903,4,4,1,0.00,43.0,36.0


Now, we will normalize the data.

In [7]:
# should only be computed using training data
mean = merged.mean(axis=0)
merged -= mean
std = merged.std(axis=0)
merged /= std
merged.head()

,pal_mean,weekday,week,month,PRCP,TMAX,TMIN
2009-02-27,-0.215528,0.500249,-1.154046,-1.298471,-0.256322,-0.112321,-0.349931
2014-06-03,0.651304,-0.999955,-0.226536,-0.142107,-0.058958,1.317891,0.973849
2013-08-21,1.767004,-0.499887,0.502222,0.436075,-0.355005,1.482915,1.395052
2013-07-10,2.865540,-0.499887,0.104718,0.146984,-0.355005,1.207874,1.575567
2017-01-27,-0.339435,0.500249,-1.485299,-1.587562,-0.355005,-1.102468,-0.771134


In [8]:
train_df = merged.sample(frac=0.8)
labels = train_df['pal_mean'].tolist()
train_df.drop(columns=['pal_mean'], inplace=True)
train_df.head()

,weekday,week,month,PRCP,TMAX,TMIN
2011-12-19,-1.500023,1.628484,1.592439,-0.355005,-0.882435,-1.252508
2009-03-30,-1.500023,-0.822792,-1.009380,-0.355005,-0.552386,-0.470274
2011-02-28,-1.500023,-1.154046,-1.298471,0.730500,-0.167329,-0.530446
2018-02-03,1.000318,-1.419048,-1.298471,-0.355005,-1.762565,-1.974570
2017-10-02,-1.500023,0.899726,1.014257,-0.355005,0.547777,0.191616


In [9]:
main_input = Input(shape=(train_df.shape[1],), name='main_input')

x = Dense(64, activation='relu')(main_input)
x = Dropout(.25)(x)
x = Dense(64, activation='relu')(x)

output = Dense(1, name='output')(x)

model = Model(inputs=[main_input], outputs=output)
model.summary()

INFO:plaidml:Opening device "metal_amd_radeon_rx_580.0"


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 6)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                448       
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
output (Dense)               (None, 1)                 65        
Total params: 4,673
Trainable params: 4,673
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['mae'])
history = model.fit([train_df], labels,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2,
                   )

Train on 3528 samples, validate on 882 samples
Epoch 1/50
3528/3528 [==============================] - 1s 152us/step - loss: 0.3623 - mean_absolute_error: 0.4515 - val_loss: 0.1360 - val_mean_absolute_error: 0.2742
Epoch 2/50
3528/3528 [==============================] - 0s 124us/step - loss: 0.1732 - mean_absolute_error: 0.3061 - val_loss: 0.1178 - val_mean_absolute_error: 0.2460
Epoch 3/50
3528/3528 [==============================] - 0s 118us/step - loss: 0.1552 - mean_absolute_error: 0.2860 - val_loss: 0.1074 - val_mean_absolute_error: 0.2396
Epoch 4/50
3528/3528 [==============================] - 0s 111us/step - loss: 0.1366 - mean_absolute_error: 0.2707 - val_loss: 0.1064 - val_mean_absolute_error: 0.2319
Epoch 5/50
3528/3528 [==============================] - 0s 110us/step - loss: 0.1224 - mean_absolute_error: 0.2582 - val_loss: 0.1004 - val_mean_absolute_error: 0.2321
Epoch 6/50
3528/3528 [==============================] - 0s 110us/step - loss: 0.1254 - mean_absolute_error: 0.257

In [11]:
source = pd.DataFrame(history.history)
source['epoch'] = source.index + 1
acc_plot = figure(plot_width=800, plot_height=400, x_axis_label='Epoch', y_axis_label='MAE')
acc_plot.circle(x='epoch', y='mean_absolute_error', source=source, size=10, fill_alpha=.5, legend_label='MAE')
acc_plot.line(x='epoch', y='val_mean_absolute_error', source=source, line_width=2, legend_label='Val MAE', alpha=0.5)
show(acc_plot)

In [12]:
loss_plot = figure(plot_width=800, plot_height=400, x_axis_label='Epoch', y_axis_label='Loss')
loss_plot.circle(x='epoch', y='loss', source=source, size=10, fill_alpha=.5, legend_label='Loss')
loss_plot.line(x='epoch', y='val_loss', source=source, line_width=2, legend_label='Val Loss', alpha=0.5)
show(loss_plot)

In [13]:
predictions = model.predict(merged.copy().drop(columns=['pal_mean']))
results = merged.copy()
results['prediction'] = predictions
results['pal_mean'] *= std.pal_mean
results['pal_mean'] += mean.pal_mean
results['prediction'] *= std.pal_mean
results['prediction'] += mean.pal_mean
results['date'] = results.index
results

,pal_mean,weekday,week,month,PRCP,TMAX,TMIN,prediction,date
2009-02-27,5893.711765,0.500249,-1.154046,-1.298471,-0.256322,-0.112321,-0.349931,5509.038086,2009-02-27
2014-06-03,6732.903082,-0.999955,-0.226536,-0.142107,-0.058958,1.317891,0.973849,7070.421387,2014-06-03
2013-08-21,7813.025424,-0.499887,0.502222,0.436075,-0.355005,1.482915,1.395052,8022.878906,2013-08-21
2013-07-10,8876.531724,-0.499887,0.104718,0.146984,-0.355005,1.207874,1.575567,8013.886230,2013-07-10
2017-01-27,5773.755903,0.500249,-1.485299,-1.587562,-0.355005,-1.102468,-0.771134,5719.138184,2017-01-27
...,...,...,...,...,...,...,...,...,...
2006-09-05,6174.697674,-0.999955,0.634723,0.725166,0.779842,0.327744,0.733162,6287.090820,2006-09-05
2013-04-15,5483.545486,-1.500023,-0.690291,-0.720289,-0.355005,-0.222337,-0.289759,5473.591797,2013-04-15
2014-09-01,7406.609247,-1.500023,0.634723,0.725166,-0.355005,1.372899,1.575567,7962.198242,2014-09-01
2019-05-14,5299.572069,-0.999955,-0.425288,-0.431198,0.385112,-0.607394,-0.289759,5537.526367,2019-05-14


In [14]:
results = results.sort_index()
# results = results[-100:]
results

,pal_mean,weekday,week,month,PRCP,TMAX,TMIN,prediction,date
2005-02-01,5580.662630,-0.999955,-1.419048,-1.298471,-0.355005,-1.267492,-1.433024,5971.286133,2005-02-01
2005-02-02,5535.835517,-0.499887,-1.419048,-1.298471,-0.355005,-1.267492,-1.252508,5943.454102,2005-02-02
2005-02-03,5514.954639,0.000181,-1.419048,-1.298471,-0.305664,-1.212484,-1.192336,5940.933105,2005-02-03
2005-02-04,5717.773469,0.500249,-1.419048,-1.298471,0.311101,-0.937443,-0.891477,5712.355957,2005-02-04
2005-02-05,5130.559122,1.000318,-1.419048,-1.298471,-0.355005,-0.552386,-0.650790,5158.868164,2005-02-05
...,...,...,...,...,...,...,...,...,...
2020-03-26,4709.563448,0.000181,-0.889043,-1.009380,-0.355005,-0.167329,-0.650790,5568.157715,2020-03-26
2020-03-27,4544.689236,0.500249,-0.889043,-1.009380,-0.355005,0.327744,0.071272,5460.233887,2020-03-27
2020-03-28,4425.012329,1.000318,-0.889043,-1.009380,0.755171,-0.497378,-0.289759,4976.417480,2020-03-28
2020-03-29,4447.838255,1.500386,-0.889043,-1.009380,-0.231652,-0.882435,-0.289759,4859.572266,2020-03-29


In [15]:
# results = ColumnDataSource(results)
pred_plot = figure(plot_width=1200, plot_height=600, x_axis_label='Date', y_axis_label='Usage')
pred_plot.circle(x='date', y='pal_mean', source=results, size=10, fill_alpha=.5, legend_label='Actual')
pred_plot.triangle(x='date', y='prediction', source=results, size=10, fill_alpha=.5, legend_label='Prediction', color='green')
pred_plot.line(x='date', y='prediction', source=results, alpha=.5, legend_label='Prediction', color='green')
pred_plot.xaxis.formatter=DatetimeTickFormatter()
show(pred_plot)